# Chunking NER

In [3]:
import nltk
nltk.download('conll2000')
from nltk.corpus import conll2000

[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.


## Exercise 1: Chunking

Let's do some data exploration.
1. First, how many sentences are there?
2. How many NP chunks?
3. How many VP chunks?
4. How many PP chunks?
5. What is the average length of each?

In [16]:
sent_99 = conll2000.chunked_sents('train.txt')[99]
print(sent_99)
print(sent_99.flatten())

(S
  (PP Over/IN)
  (NP a/DT cup/NN)
  (PP of/IN)
  (NP coffee/NN)
  ,/,
  (NP Mr./NNP Stone/NNP)
  (VP told/VBD)
  (NP his/PRP$ story/NN)
  ./.)
(S
  Over/IN
  a/DT
  cup/NN
  of/IN
  coffee/NN
  ,/,
  Mr./NNP
  Stone/NNP
  told/VBD
  his/PRP$
  story/NN
  ./.)


In [28]:
sents = conll2000.chunked_sents('train.txt')
np_count, vp_count, pp_count = 0, 0, 0
np_len, vp_len, pp_len = 0, 0, 0
for sent in sents:
    for subtree in sent.subtrees():
        label = subtree.label()
        if label == "NP":
            np_count += 1
            np_len += len(subtree)
        if label == "VP":
            vp_count += 1
            vp_len += len(subtree)
        if label == "PP":
            pp_count += 1
            pp_len += len(subtree)
np_avg = round(np_len / np_count, 2)
vp_avg = round(vp_len / vp_count, 2)
pp_avg = round(pp_len / pp_count, 2)
print("Sentences:", len(sents))
print("NP Chunks:", np_count)
print("VP Chunks:", vp_count)
print("PP Chunks:", pp_count)
print("NP Chunks Avg Length:", np_avg)
print("VP Chunks Avg Length:", vp_avg)
print("PP Chunks Avg Length:", pp_avg)

Sentences: 8936
NP Chunks: 55081
VP Chunks: 21467
PP Chunks: 21281
NP Chunks Avg Length: 2.15
VP Chunks Avg Length: 1.56
PP Chunks Avg Length: 1.01


In [39]:
from nltk import FreqDist
from collections import defaultdict
import numpy as np
labels = FreqDist()
lengths = defaultdict(lambda: list())

sents = conll2000.chunked_sents('train.txt')
for sent in sents:
    for subtree in sent.subtrees():
        label = subtree.label()
        labels.update([label])
        lengths[label].append(len(subtree))

print("Sentences:", len(sents))
for label in ["NP", "VP", "PP"]:
    print(label)
    print("Chunks:", labels[label])
    print("Length:", round(np.mean(lengths[label]), 2))

Sentences: 8936
NP
Chunks: 55081
Length: 2.15
VP
Chunks: 21467
Length: 1.56
PP
Chunks: 21281
Length: 1.01


## Exercise 2: Unigram chunker

Now, let's concentrate only on NP chunking
1. Create a unigram chunker using the UnigramChunker class below.
Train on the train sentences and evaluate on the test sentences using
the evaluate method, i.e., my_model.evaluate(test_sents).

2. What is the F1 score?

In [25]:
class UnigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.UnigramTagger(train_data)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)


train_sents = conll2000.chunked_sents('train.txt', chunk_types=['NP'])
test_sents = conll2000.chunked_sents('test.txt', chunk_types=['NP'])

unigram_chunker = UnigramChunker(train_sents)
print(unigram_chunker.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  92.9%%
    Precision:     79.9%%
    Recall:        86.8%%
    F-Measure:     83.2%%


## Exercise 3: Bigram/Trigram chunker

Now, modify the code to create Bigram and Trigram taggers

In [26]:
class BigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.BigramTagger(train_data)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)
        
bigram_chunker = BigramChunker(train_sents)
print(bigram_chunker.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  93.3%%
    Precision:     82.3%%
    Recall:        86.8%%
    F-Measure:     84.5%%


In [32]:
class TrigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.TrigramTagger(train_data)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)
        
trigram_chunker = TrigramChunker(train_sents)
print(trigram_chunker.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  93.3%%
    Precision:     82.5%%
    Recall:        86.8%%
    F-Measure:     84.6%%


## Exercise 4: Maximum Entropy model with features

Finally, we will use a maximum entropy classifier (a discriminative classifier)
to model the chunking task. Remember that discriminative classifiers attempt to
model p(y|x) directly, which allows us more freedom in what x is.

In [35]:
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    return {"pos": pos}


class ConsecutiveNPChunkTagger(nltk.TaggerI):

    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = npchunk_features(untagged_sent, i, history)
                train_set.append((featureset, tag))
                history.append(tag)
        self.classifier = nltk.MaxentClassifier.train(
            train_set, max_iter=10, trace=0)

    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = npchunk_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)


class ConsecutiveNPChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        tagged_sents = [[((w, t), c) for (w, t, c) in
                         nltk.chunk.tree2conlltags(sent)]
                        for sent in train_sents]
        self.tagger = ConsecutiveNPChunkTagger(tagged_sents)

    def parse(self, sentence):
        tagged_sents = self.tagger.tag(sentence)
        conlltags = [(w,t,c) for ((w,t),c) in tagged_sents]
        return nltk.chunk.conlltags2tree(conlltags)

In [36]:
consecutive_chunker = ConsecutiveNPChunker(train_sents)
print(consecutive_chunker.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  92.9%%
    Precision:     79.9%%
    Recall:        86.8%%
    F-Measure:     83.2%%


## Exercise 5: Add more features to get better performance

Add in more features to get better performance

In [40]:
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    if i == 0:
        prevword, prevpos = "<START>", "<START>"
    else:
        prevword, prevpos = sentence[i-1]
    if i == len(sentence)-1:
        nextword, nextpos = "<END>", "<END>"
    else:
        nextword, nextpos = sentence[i+1]
    features = {
        "pos": pos,
        "word": word,
        "prevpos": prevpos,
        "prevword": prevword,
        "nextpos": nextpos,
        "nextword": nextword,
    }
    return features

In [41]:
consecutive_chunker = ConsecutiveNPChunker(train_sents)
print(consecutive_chunker.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  95.6%%
    Precision:     87.8%%
    Recall:        91.0%%
    F-Measure:     89.4%%


## Exercise 6: NER Data

We will use a dataset of tweets annotated for Named Entities.

```
@inproceedings{derczynski-etal-2016-broad,
    title = "Broad {T}witter Corpus: A Diverse Named Entity Recognition Resource",
    author = "Derczynski, Leon  and
      Bontcheva, Kalina  and
      Roberts, Ian",
    booktitle = "Proceedings of {COLING} 2016, the 26th International Conference on Computational Linguistics: Technical Papers",
    month = dec,
    year = "2016",
    address = "Osaka, Japan",
    publisher = "The COLING 2016 Organizing Committee",
    url = "https://aclanthology.org/C16-1111",
    pages = "1169--1179",
    abstract = "One of the main obstacles, hampering method development and comparative evaluation of named entity recognition in social media, is the lack of a sizeable, diverse, high quality annotated corpus, analogous to the CoNLL{'}2003 news dataset. For instance, the biggest Ritter tweet corpus is only 45,000 tokens {--} a mere 15{\%} the size of CoNLL{'}2003. Another major shortcoming is the lack of temporal, geographic, and author diversity. This paper introduces the Broad Twitter Corpus (BTC), which is not only significantly bigger, but sampled across different regions, temporal periods, and types of Twitter users. The gold-standard named entity annotations are made by a combination of NLP experts and crowd workers, which enables us to harness crowd recall while maintaining high quality. We also measure the entity drift observed in our dataset (i.e. how entity representation varies over time), and compare to newswire. The corpus is released openly, including source text and intermediate annotations.",
}
```

In [ ]:
# load the data (1000 annotated tweets)
import json
from sklearn.metrics import f1_score

ner_data = []
for line in open("twitter_NER.json"):
    ner_data.append(json.loads(line))


# each example in ner_data is a json dictionary that contains two values we are interested in: tokens, entities

print(ner_data[6]["tokens"])
print(ner_data[6]["entities"])

# we need the training data as a list of lists, where the inner list contains tuples of (token, label) i.e., [[(token_1, label_1 ), (token_2, label_2), ...]]

# Test data should be a list of lists with the inner list having tokens

# Test labels should be a flat list of labels ['O', 'B-PER', 'I-PER', 'O'...]

ner_train_data = []
ner_test_data = []
ner_test_labels = []
for s in ner_data[:800]:
    ner_train_data.append(list(zip(s["tokens"], s["entities"])))
for s in ner_data[800:]:
    ner_test_data.append(s["tokens"])
    ner_test_labels.extend(s["entities"])

## Exercise 7: Hidden Markov Model

In [ ]:
tagger = nltk.HiddenMarkovModelTagger.train(ner_train_data)


# Evaluate the model using the f1_score function from sklearn.metrics
# You should use macro F1 and make sure NOT TO COUNT the 'O' label

f1 = 0.00
print("F1 score: {0:.3f}".format(f1))

## Exercise 7: Optional further exercises

The previous F1 is calculated at token level. However, for NER, we often calculate F1 at entity level.

Implement your own code to evaluate entity-level NER

1. You will need to calculate:
    Precision = (number of correctly predicted entities) / (number of predicted entities)

    Recall = (number of correctly predicted entites) / (number of gold entities)

    F1 = (2 * Precision * Recall) / (Precision + Recall)


2. You might want to implement a helper function which, given labels in IOB2 format return a list of all the entities.
